In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input
from keras.callbacks import EarlyStopping
from keras.models import model_from_json
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import joblib

In [2]:
import pandas as pd 
data = pd.read_csv('ckextended.csv')
data.head()

,emotion,pixels,Usage
0,6,36 39 35 25 19 11 8 7 3 13 15 9 21 57 75 90 10...,Training
1,6,88 74 19 4 5 5 3 12 8 21 15 21 15 18 24 29 32 ...,Training
2,6,9 2 4 7 1 1 1 0 7 29 49 76 115 141 156 169 177...,Training
3,6,104 106 108 104 95 50 60 61 58 83 126 133 139 ...,Training
4,6,68 72 67 67 6 2 1 1 1 1 1 14 24 24 38 65 79 94...,Training


In [3]:
data['emotion'].unique()

array([6, 3, 7, 0, 1, 4, 2, 5])

In [4]:
data.drop(columns=['Usage'],axis=-1,inplace=True)

In [5]:
data.head()

,emotion,pixels
0,6,36 39 35 25 19 11 8 7 3 13 15 9 21 57 75 90 10...
1,6,88 74 19 4 5 5 3 12 8 21 15 21 15 18 24 29 32 ...
2,6,9 2 4 7 1 1 1 0 7 29 49 76 115 141 156 169 177...
3,6,104 106 108 104 95 50 60 61 58 83 126 133 139 ...
4,6,68 72 67 67 6 2 1 1 1 1 1 14 24 24 38 65 79 94...


In [6]:
import numpy as np 
def process_pixels(pixels_series):
    images = []
    for pixel_seq in pixels_series:
        pixel_vals = np.array([int(p) for p in pixel_seq.split()],dtype='float32')
        image = pixel_vals.reshape(48,48,1)/255.0
        images.append(image)
    return np.array(images)


In [7]:
x_data = process_pixels(data['pixels'])
y_data = data['emotion'].values

In [8]:
x_data.shape

(920, 48, 48, 1)

In [9]:
y_cat = to_categorical(y_data, num_classes=8)

In [10]:
y_cat

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [11]:
x_train, x_test, y_train, y_test, y_train_labels, y_test_labels = train_test_split(
    x_data, y_cat, y_data, test_size=0.2, random_state=42
)
cnn_model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu', name='feature_layer'),
    Dropout(0.5),
    Dense(8, activation='softmax')  
])
cnn_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history = cnn_model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=40,
    validation_data=(x_test, y_test),
)

Epoch 1/40


/Users/nabarundutta/Downloads/Projects/emotion/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - accuracy: 0.4848 - loss: 1.7067 - val_accuracy: 0.6413 - val_loss: 1.4228
Epoch 2/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 0.6257 - loss: 1.4307 - val_accuracy: 0.6413 - val_loss: 1.5329
Epoch 3/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.6576 - loss: 1.3351 - val_accuracy: 0.6413 - val_loss: 1.5089
Epoch 4/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.6469 - loss: 1.3334 - val_accuracy: 0.6413 - val_loss: 1.4490
Epoch 5/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.6635 - loss: 1.3145 - val_accuracy: 0.6413 - val_loss: 1.4888
Epoch 6/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.6262 - loss: 1.3664 - val_accuracy: 0.6413 - val_loss: 1.3377
Epoch 7/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.6249 - loss: 1.4331 - val_accuracy: 0.6413 - val_loss: 1.3449
Epoch 8/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.6477 - loss: 1.3408 - val_accuracy: 0.6413 - val_los

In [12]:
feature_extractor = Model(inputs=cnn_model.layers[0].input, outputs=cnn_model.get_layer('feature_layer').output)
x_train_feat = feature_extractor.predict(x_train)
x_test_feat = feature_extractor.predict(x_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [13]:
svm = SVC(kernel='linear')
svm.fit(x_train_feat, y_train_labels)
y_pred_svm = svm.predict(x_test_feat)

print("SVM Accuracy on CNN features:", accuracy_score(y_test_labels, y_pred_svm))

SVM Accuracy on CNN features: 0.8804347826086957


In [14]:
y_test_labels

array([6, 6, 6, 6, 6, 6, 6, 7, 4, 6, 6, 5, 6, 3, 2, 6, 6, 6, 6, 1, 6, 6,
       6, 6, 0, 0, 3, 6, 5, 6, 6, 6, 6, 6, 6, 1, 6, 6, 6, 6, 0, 6, 3, 6,
       6, 1, 3, 6, 6, 6, 1, 6, 6, 6, 6, 1, 0, 6, 6, 6, 6, 6, 7, 0, 6, 5,
       6, 5, 6, 4, 4, 6, 1, 6, 6, 6, 6, 2, 6, 6, 3, 5, 6, 5, 5, 5, 4, 6,
       6, 6, 1, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 5, 6, 6,
       6, 6, 6, 6, 5, 6, 1, 6, 6, 6, 6, 2, 6, 6, 0, 5, 3, 6, 3, 0, 6, 6,
       6, 6, 1, 6, 6, 4, 6, 4, 6, 5, 5, 3, 2, 6, 5, 1, 6, 7, 1, 0, 6, 5,
       6, 2, 6, 6, 5, 6, 6, 5, 6, 6, 3, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 1, 3, 6, 6, 5, 6, 1])

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train_feat,y_train_labels)
y_pred = rf.predict(x_test_feat)
accuracy_score(y_test_labels,y_pred)

0.8967391304347826

In [16]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(x_train_feat,y_train_labels)
y_pred_ada = rf.predict(x_test_feat)
accuracy_score(y_test_labels,y_pred_ada)

0.8967391304347826